In [1]:
from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
from evaluations.clustering import *  # I’m so sorry

In [ ]:
GD['joliet']

In [ ]:
# base_dir = '../../results/only remove deno BS128'
# base_dir = '../../results/cono space remove deno/subset pretrained'
base_dir = '../../results/deno space remove cono/superset pretrained'
models = load_en_masse(base_dir, endswith='epoch100.pt')
models['pretrained superset'] = load('../../results/pretrained/init.pt')
models['pretrained'] = load('../../results/pretrained bill mentions/init.pt')

### Graph by Party Skew (for removing connotation)

In [ ]:
graph_en_masse(
    models,
    out_dir=f'{base_dir}/PCA',
    reduction='PCA',
    word_ids=R_ids,
    words=R_words,
    hues=R_skew,
    sizes=R_freq,
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/t-SNE p5',
    reduction='TSNE',
    perplexity=5,
    word_ids=R_ids,
    words=R_words,
    hues=R_skew,
    sizes=R_freq,
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/t-SNE p3',
    reduction='TSNE',
    perplexity=3,
    word_ids=R_ids,
    words=R_words,
    hues=R_skew,
    sizes=R_freq,
)

In [ ]:
# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/PCA',
#     reduction='PCA',
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_skew,
#     sizes=J_freq,
# )

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/t-SNE p5',
#     reduction='TSNE',
#     perplexity=5,
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_skew,
#     sizes=J_freq,
# )

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/t-SNE p3',
#     reduction='TSNE',
#     perplexity=3,
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_skew,
#     sizes=J_freq,
# )

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/t-SNE p25',
    reduction='TSNE',
    perplexity=25,
    word_ids=J_ids,
    words=J_words,
    hues=J_skew,
    sizes=J_freq,
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/t-SNE p50',
    reduction='TSNE',
    perplexity=50,
    word_ids=J_ids,
    words=J_words,
    hues=J_skew,
    sizes=J_freq,
)

### Graph by Topic Denotation (for removing denotation)

In [ ]:
graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly GOP/PCA',
    reduction='PCA',
    perplexity=5,
    word_ids=GOP_ids,
    words=GOP_words,
    hues=GOP_deno,
    sizes=GOP_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly GOP/t-SNE p5',
    reduction='TSNE',
    perplexity=5,
    word_ids=GOP_ids,
    words=GOP_words,
    hues=GOP_deno,
    sizes=GOP_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly GOP/t-SNE p3',
    reduction='TSNE',
    perplexity=3,
    word_ids=GOP_ids,
    words=GOP_words,
    hues=GOP_deno,
    sizes=GOP_freq,
    categorical=True
)

In [ ]:
graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly Dem/PCA',
    reduction='PCA',
    perplexity=5,
    word_ids=D_ids,
    words=D_words,
    hues=D_deno,
    sizes=D_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly Dem/t-SNE p5',
    reduction='TSNE',
    perplexity=5,
    word_ids=D_ids,
    words=D_words,
    hues=D_deno,
    sizes=D_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Highly Dem/t-SNE p3',
    reduction='TSNE',
    perplexity=3,
    word_ids=D_ids,
    words=D_words,
    hues=D_deno,
    sizes=D_freq,
    categorical=True
)

In [ ]:
# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/PCA',
#     reduction='PCA',
#     perplexity=5,
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_deno,
#     sizes=J_freq,
#     categorical=True
# )

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/t-SNE p5',
    reduction='TSNE',
    perplexity=5,
    word_ids=J_ids,
    words=J_words,
    hues=J_deno,
    sizes=J_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/t-SNE p3',
    reduction='TSNE',
    perplexity=3,
    word_ids=J_ids,
    words=J_words,
    hues=J_deno,
    sizes=J_freq,
    categorical=True
)

## Graph Recomposers
Want to show that...

For deno vectors, topic cluster better than pretrained

For cono vectors, skew cluster better than pretrained

In [ ]:
base_dir = '../../results/recomposer/superset pretrained'
D_sub, C_sub = load_recomposers_en_masse(base_dir, endswith='epoch30.pt')

In [5]:
base_dir = '../../results/recomposer/superset pretrained/dumb recomposer'
D_sub, C_sub = load_recomposers_en_masse(base_dir, endswith='epoch30.pt')

/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'decomposer.Decomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'recomposer.Recomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


['Dd1.9', 'Dg-0.2', 'Cd-1.3', 'Cg3.0', 'R1', 'epoch30.pt']
['Dd1.9', 'Dg-0.2', 'Cd-1.3', 'Cg3.0', 'R4.0', 'not', 'dumb', 'epoch30.pt']
['Dd1.9', 'Dg-0.2', 'Cd-1.3', 'Cg3.0', 'R4.0', 'epoch30.pt']


In [ ]:
base_dir = '../../results/recomposer/superset pretrained'
D_super, C_super = load_recomposers_en_masse(base_dir, endswith='epoch30.pt')

In [6]:
 # Evaluating Denotation
models = D_sub

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/topic/PCA',
#     reduction='PCA',
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_deno,
#     sizes=J_freq,
#     categorical=True
# )

graph_en_masse(
    models, out_dir=f'{base_dir}/Joint/topic/t-SNE p5',
    reduction='TSNE', perplexity=5,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/topic/t-SNE p3',
    reduction='TSNE', perplexity=3,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/topic/t-SNE p10',
    reduction='TSNE', perplexity=10,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True
)

100%|██████████| 5/5 [00:14<00:00,  2.80s/it]


In [7]:
# Evaluating Connotation
models = C_sub

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/topic/PCA',
#     reduction='PCA',
#     word_ids=J_ids,
#     words=J_words,
#     hues=J_skew,
#     sizes=J_freq,
# )


graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq,
)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/Joint/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq,
)

100%|██████████| 5/5 [00:09<00:00,  1.97s/it]


# Clustering + Homogeneity V-Measure

In [ ]:
# Deno space, eval deno, higher is better
for model_name, model in D_super.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=10)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Deno space, eval cono, lower is better
for model_name, model in D_super.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Cono space, eval cono, higher is better
for model_name, model in C_super.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')

In [ ]:
# Cono space, eval deno, lower is better
for model_name, model in C_super.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')

In [ ]:
# homogeneity, completeness, v_measure = np.around(
#     homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
# print(homogeneity, completeness, v_measure, sep='\t')